1. найти число событий в конкретный день

In [1]:
import pandas as pd
from datetime import datetime, timedelta

def count_sessions():
    '''Функция подсчета количества сессий'''
    # Открыть файл для чтения
    with open('log.csv', 'r') as f:
        # Чтение файла формата csv
        df = pd.read_csv(f)
        # Преобразование времени в формат datetime
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d_%H:%M:%S')
        # Сортировка по пользователю и времени
        df = df.sort_values(['user', 'date'])
        # Вычисление разницы между событиями пользователя
        df['time_diff'] = df.groupby('user')['date'].diff()
        # Определяем начало новых сессий (разница >30 минут или первый эвент пользователя)
        df['new_session'] = (df['time_diff'].isna()) | (df['time_diff'] > timedelta(minutes=30))
        # Считаем сессии, начавшиеся 2020-04-19
        target_date = pd.to_datetime('2020-04-19').date()
        # Условие на новую сессию пользователя в нужную дату
        sessions_count = df[df['new_session'] & (df['date'].dt.date == target_date)].shape[0]
        return sessions_count

In [2]:
print(count_sessions())

5256
